In [1]:
import os
import sys
import torch
p = os.path.join(os.path.abspath("../.."), "main")
sys.path.insert(1, p)

from models.diffusion.classifier import mobilenet_v2
from models.diffusion.gcr import GCR
import hydra
from util import dotdict
from models.vae import VAE
from models.diffusion import DDPM, DDPMv2, DDPMWrapper, SuperResModel, UNetModel
from datasets import CIFAR10Dataset
from LACE.sampling import _sample_q_dict
from LACE.eval_sampler import ConditionalSampling





state_dict_clf = torch.load("/Users/gio/Desktop/UIC/CS594/Class project/GRC/main/models/diffusion/state_dicts/mobilenet_v2.pt")
clf = mobilenet_v2(pretrained=True, progress=True, device="cpu")
clf.load_state_dict(state_dict_clf)
clf.eval()



vae = VAE.load_from_checkpoint(
        "/Users/gio/Desktop/UIC/CS594/Class project/GRC/main/checkpoints/vae_cifar10_loss=0.00.ckpt",
        input_res=32,
    )
vae.eval()


config = dotdict({'dataset': dotdict({'ddpm': dotdict({'data': dotdict({'root': '???', 'name': 'cifar10', 'image_size': 32, 'hflip': True, 'n_channels': 3, 'norm': True, 'ddpm_latent_path': ''}), 'model': dotdict({'dim': 160, 'attn_resolutions': '16,', 'n_residual': 3, 'dim_mults': '1,2,2,2', 'dropout': 0.3, 'n_heads': 8, 'beta1': 0.0001, 'beta2': 0.02, 'n_timesteps': 1000}), 'evaluation': dotdict({'chkpt_path': '/Users/gio/Desktop/UIC/CS594/Class project/GRC/main/checkpoints/diffvae_cifar10_form1_largermodel_loss=0.0438.ckpt', 'save_path': '.', 'z_cond': False, 'z_dim': 512, 'guidance_weight': 0.0, 'type': 'form1', 'resample_strategy': 'truncated', 'skip_strategy': 'quad', 'sample_method': 'ddpm', 'sample_from': 'target', 'seed': 0, 'device': 'cpu', 'n_samples': 2500, 'n_steps': 1000, 'workers': 4, 'batch_size': 2, 'save_vae': True, 'variance': 'fixedlarge', 'sample_prefix': 'gpu_1', 'temp': 1.0, 'save_mode': 'image'}), 'interpolation': dotdict({'n_steps': 10})}), 'vae': dotdict({'data': dotdict({'root': '???', 'name': 'cifar10', 'image_size': 32, 'n_channels': 3}), 'model': dotdict({'z_dim': 512, 'enc_block_config': '32x7,32d2,32t16,16x4,16d2,16t8,8x4,8d2,8t4,4x3,4d4,4t1,1x3', 'enc_channel_config': '32:64,16:128,8:256,4:256,1:512', 'dec_block_config': '1x1,1u4,1t4,4x2,4u2,4t8,8x3,8u2,8t16,16x7,16u2,16t32,32x15', 'dec_channel_config': '32:64,16:128,8:256,4:256,1:512'}), 'evaluation': dotdict({'chkpt_path': '/Users/gio/Desktop/UIC/CS594/Class project/GRC/main/checkpoints/vae_cifar10_loss=0.00.ckpt', 'save_path': '???', 'expde_model_path': '', 'seed': 0, 'device': 'gpu:0', 'workers': 2, 'batch_size': 8, 'n_samples': 50000, 'sample_prefix': '', 'save_mode': 'image'})})})})

In [2]:
import copy

import hydra
import pytorch_lightning as pl
import torch
from datasets.latent import LatentDataset
from models.callbacks import ImageWriter
from models.diffusion import DDPM, DDPMv2, DDPMWrapper, SuperResModel
from models.vae import VAE
from pytorch_lightning.utilities.seed import seed_everything
from torch.utils.data import DataLoader
from util import configure_device


def __parse_str(s):
    split = s.split(",")
    return [int(s) for s in split if s != "" and s is not None]



# Seed and setup
config_ddpm = config.dataset.ddpm
config_vae = config.dataset.vae
seed_everything(config_ddpm.evaluation.seed, workers=True)

batch_size = 256#config_ddpm.evaluation.batch_size
n_steps = config_ddpm.evaluation.n_steps
n_samples = config_ddpm.evaluation.n_samples
image_size = config_ddpm.data.image_size
ddpm_latent_path = config_ddpm.data.ddpm_latent_path
ddpm_latents = torch.load(ddpm_latent_path) if ddpm_latent_path != "" else None

# Load pretrained VAE
vae = VAE.load_from_checkpoint(
    config_vae.evaluation.chkpt_path,
    input_res=image_size,
)
vae.eval()

# Load pretrained wrapper
attn_resolutions = __parse_str(config_ddpm.model.attn_resolutions)
dim_mults = __parse_str(config_ddpm.model.dim_mults)
decoder = SuperResModel(
    in_channels=config_ddpm.data.n_channels,
    model_channels=config_ddpm.model.dim,
    out_channels=3,
    num_res_blocks=config_ddpm.model.n_residual,
    attention_resolutions=attn_resolutions,
    channel_mult=dim_mults,
    use_checkpoint=False,
    dropout=config_ddpm.model.dropout,
    num_heads=config_ddpm.model.n_heads,
    z_dim=config_ddpm.evaluation.z_dim,
    use_scale_shift_norm=config_ddpm.evaluation.z_cond,
    use_z=config_ddpm.evaluation.z_cond,
)

ema_decoder = copy.deepcopy(decoder)
decoder.eval()
ema_decoder.eval()

ddpm_cls = DDPMv2 if config_ddpm.evaluation.type == "form2" else DDPM
online_ddpm = ddpm_cls(
    decoder,
    beta_1=config_ddpm.model.beta1,
    beta_2=config_ddpm.model.beta2,
    T=config_ddpm.model.n_timesteps,
    var_type=config_ddpm.evaluation.variance,
)
target_ddpm = ddpm_cls(
    ema_decoder,
    beta_1=config_ddpm.model.beta1,
    beta_2=config_ddpm.model.beta2,
    T=config_ddpm.model.n_timesteps,
    var_type=config_ddpm.evaluation.variance,
)

ddpm_wrapper = DDPMWrapper.load_from_checkpoint(
    config_ddpm.evaluation.chkpt_path,
    online_network=online_ddpm,
    target_network=target_ddpm,
    vae=vae,
    conditional=True,
    pred_steps=n_steps,
    eval_mode="sample",
    resample_strategy=config_ddpm.evaluation.resample_strategy,
    skip_strategy=config_ddpm.evaluation.skip_strategy,
    sample_method=config_ddpm.evaluation.sample_method,
    sample_from=config_ddpm.evaluation.sample_from,
    data_norm=config_ddpm.data.norm,
    temp=config_ddpm.evaluation.temp,
    guidance_weight=config_ddpm.evaluation.guidance_weight,
    z_cond=config_ddpm.evaluation.z_cond,
    strict=True,
    ddpm_latents=ddpm_latents,
)

# Create predict dataset of latents
z_dataset = LatentDataset(
    (n_samples, config_vae.model.z_dim, 1, 1),
    (n_samples, 3, image_size, image_size),
    share_ddpm_latent=True if ddpm_latent_path != "" else False,
    expde_model_path=config_vae.evaluation.expde_model_path,
    seed=config_ddpm.evaluation.seed,
)

# Setup devices
test_kwargs = {}
loader_kws = {}
device = config_ddpm.evaluation.device
if device.startswith("gpu"):
    _, devs = configure_device(device)
    test_kwargs['accelerator'] = "gpu"
    test_kwargs["gpus"] = devs

    # Disable find_unused_parameters when using DDP training for performance reasons
    loader_kws["persistent_workers"] = True
elif device == "tpu":
    test_kwargs["tpu_cores"] = 8

# Predict loader
#pin_memory commented
val_loader = DataLoader(
    z_dataset,
    batch_size=batch_size,
    drop_last=False,
    shuffle=False,
    num_workers=config_ddpm.evaluation.workers,
    **loader_kws,
)

# Predict trainer
write_callback = ImageWriter(
    config_ddpm.evaluation.save_path,
    "batch",
    n_steps=n_steps,
    eval_mode="sample",
    conditional=True,
    sample_prefix=config_ddpm.evaluation.sample_prefix,
    save_vae=config_ddpm.evaluation.save_vae,
    save_mode=config_ddpm.evaluation.save_mode,
    is_norm=config_ddpm.data.norm,
)




/Users/gio/opt/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
Global seed set to 0


In [3]:
model = GCR(ddpm_wrapper, clf, config_vae.model.z_dim)

sample_q = _sample_q_dict["sample_q_sgld"]
c = torch.ones((1), dtype=torch.int64)
ode_kwargs = {'atol': 1e-3, 'rtol': 1e-3, 'method': "dopri5", 'use_adjoint': True}
ld_kwargs = {'batch_size': batch_size, 'latent_dim': config_vae.model.z_dim, 'sgld_lr': 1,
                'sgld_std': 1e-2, 'n_steps': 20}
sde_kwargs = {'N': 1000, 'correct_nsteps': 2, 'target_snr': 0.16}

n_classes = 10
condSampling = ConditionalSampling(sample_q, batch_size, config_vae.model.z_dim, n_classes, model,
                                    device, ".", ode_kwargs, ld_kwargs, sde_kwargs,
                                    every_n_plot=1)


In [4]:
condSampling.get_samples()

In [23]:
for z_ddpm, z_vae in val_loader:
    break

KeyError: 'latent_dim'

In [44]:
model.generate_images(torch.rand((512,2,512))).shape

torch.Size([3, 2, 16384])